# Pip install ipywidgets to environment when running for first time
May need to pip install several of the other imports depending on how environment is set up

In [1]:
-m pip install ipywidgets==8.1.1
-m jupyter nbextension enable --py widgetsnbextension

SyntaxError: invalid syntax (2653151621.py, line 1)

# Importing modules and setting up classes

In [67]:
from IPython.display import clear_output, display
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import ipywidgets as widgets
import plotly.express as px
import pandas as pd
import pandas as pd
import numpy as np
import requests
import glob
import sys
import os
import re

In [75]:
# Setting up timing mappings to translate MATLAB syntax
TIMING_MAPPING = {
    "hourly": "H",
    "daily": "D",
    "minutely": "T"
}

REVERSE_TIMING_MAPPING = {v: k for k, v in TIMING_MAPPING.items()}

class CTDataProcessor:
    
    def __init__(self, timing="hourly"):
        self.timing = TIMING_MAPPING.get(timing, timing)

    def get_human_readable_timing(self):
        return REVERSE_TIMING_MAPPING.get(self.timing, self.timing)

    def process(self, user_selected_machines, filename="Results.csv"):
        # Setting up list for machines that have insufficent date
        machines_to_remove = []

        # Read CT Data
        df = self.read_CT_data(filename)        

        # get the list of deviceName
        device_names = self.list_machines(df)

        # Remove Power Failure detected rows
        df = df[df['powerFailureDetected'] != 1]

        # Export data for each machine
        for device in device_names:
            # Skip device if not in machines selected by the user
            if device not in user_selected_machines:
                continue

            # Get current data
            temp_df = df[df['deviceName'] == device][["A", "channel1", "channel2", "channel3"]]#.dropna()

            # Aggregate data
            temp_df = temp_df.resample(self.timing).mean().fillna(0)

            # Checking sufficient datapoints
            if len(temp_df) < 2:
                print(f'{device} has insufficient data to analyse on a {self.get_human_readable_timing()} basis')
                machines_to_remove.append(device)
                continue

            # Placeholder columns 
            # ARE ANY OF THESE NEEDED? CAN ADD COLUMNS AS REQURED
            temp_df["V"] = 0
            temp_df["kW"] = 0
            temp_df["cost"] = 0
            
            directory_name = str(device)
            if not os.path.exists(directory_name):
                os.makedirs(directory_name)
                
            file_name = os.path.join(directory_name, f"{self.get_human_readable_timing()}_{device}.csv")
            temp_df.to_csv(file_name)
            temp_df.to_csv('complete')
        return machines_to_remove

    def read_CT_data(self, filename):
        df = pd.read_csv(filename)
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        df.set_index('timestamp', inplace=True)
        return df#.dropna()

    def list_machines(self, df):
        devices = df['deviceName'].unique()
        return devices
    
class CTDataAnalyser:
    
    def __init__(self, machine_user_input, TIMING, REGION):
        self.machine_name = machine_user_input['Machine Name']
        self.PHASE = machine_user_input['Phase']
        self.VOLTAGE = machine_user_input['Voltage']
        self.DAY_UNIT_COST = machine_user_input['Day Unit Cost']
        self.NIGHT_UNIT_COST = machine_user_input['Night Unit Cost']
        self.NIGHT_TARIFF_START_TIME = machine_user_input['Night Tariff Start']
        self.NIGHT_TARIFF_END_TIME = machine_user_input['Night Tariff End']
        self.REGION = REGION
        self.TIMING = TIMING

    def machine_calculations(self):
        
        # Getting adjusted timescale
        self.tscale = self.adjust_timescale()

        # Loading dataframe of individual machine
        device_df = pd.read_csv(f'{self.machine_name}/{self.TIMING}_{self.machine_name}.csv')

        device_df = self.compute_kW_utilization_and_cost(device_df, self.machine_name)

        device_df = self.estimate_load_imbalance(device_df, self.machine_name)

        device_df = self.estimate_carbon_emissions(device_df)

        device_df.to_csv(f'{self.machine_name}/{self.TIMING}_{self.machine_name}.csv', index=False)

    # Calculates deviation from average between current channels for each individual current channel
    def unbalanced(self, channels):

        # Convert to numpy array for easier calculations
        channels = np.array(channels).T # Transpose to get channels as columns

        Iav = np.mean(channels, axis=1)

        u = np.max(np.abs(channels - Iav[:, np.newaxis]), axis=1) / Iav * 100
        u[np.isnan(u)] = 0
        
        return u
    
    def adjust_timescale(self):
        if self.TIMING == 'hourly':
            tscale = 1
        elif self.TIMING == 'minutely':
            tscale = 60
        elif self.TIMING == 'daily':
            tscale = 1/24
        elif self.TIMING == 'weekly':
            tscale = 1/(24*7)
        else:
            raise ValueError("Invalid timing value")
        return tscale
    
    def compute_kW_utilization_and_cost(self, device_df, device_name):
        # Correcting for error where values for A in single phase machines are stored in Channel1
        # Check if all values in column 'A' are 0 and there are non-zero values in 'Channel1'
        if device_df['A'].eq(0).all() and device_df['channel1'].ne(0).any() and self.PHASE == 1: 
            # Switch the values
            device_df['A'], device_df['channel1'] = device_df['channel1'], device_df['A']

        # Only 3-phase Machines' Current should be multiplied by sqrt(3)
        device_df['A'] = device_df['A'] * (3 ** 0.5 if self.PHASE == 3 else 1) / self.tscale

        # Insert the voltages of individual machines
        device_df['V'] = self.VOLTAGE

        # Estimate kW
        device_df['kW'] = device_df['V'] * device_df['A'] / 1000

        # Estimate Utilization
        device_df['utilization'] = (device_df['A'] > 1).astype(int)

        # Estimate unit cost of electricity
        device_df['p'] = self.DAY_UNIT_COST
        device_df['timestamp'] = pd.to_datetime(device_df['timestamp'])
        device_df.loc[device_df['timestamp'].dt.hour >= self.NIGHT_TARIFF_START_TIME, 'p'] = self.NIGHT_UNIT_COST
        device_df.loc[device_df['timestamp'].dt.hour < self.NIGHT_TARIFF_END_TIME, 'p'] = self.NIGHT_UNIT_COST
        
        device_df['cost'] = device_df['kW'] * device_df['p']
        return device_df
    
    def estimate_load_imbalance(self, device_df, device_name):
        if self.PHASE == 1:
            device_df['unbalanced'] = self.unbalanced([device_df['channel1']])
        elif self.PHASE == 3:
            device_df['unbalanced'] = self.unbalanced([device_df['channel1'], device_df['channel2'], device_df['channel3']])
        return device_df
    
    def estimate_carbon_emissions(self, device_df):
        st = str(device_df['timestamp'].iloc[0])
        en = str(device_df['timestamp'].iloc[-1])

        timestamp, co2 = self.chunk_dates_and_fetch_carbon_data(st, en)
        
        # Creating a DataFrame from the retrieved data
        tco2 = pd.DataFrame({'timestamp': timestamp, 'co2': co2})
        tco2['timestamp'] = pd.to_datetime(tco2['timestamp'])

        # Re-sampling operations
        # The MATLAB code was using 'retime' to handle the time series data.
        # `resample` and `interpolate` achieve a similar result.
        tco2.set_index('timestamp', inplace=True)
        tco2 = tco2.resample(TIMING_MAPPING.get(self.TIMING, self.TIMING)).mean().interpolate() #.reset_index()

        # With the test data tried so far this data lags an hour behind the MATLAB data, so may need to add an hour, but might an an issue with daylight savings
        # COME BACK TO THIS MUST CHECK POTENTIAL ISSUES WITH DAYLIGHT SAVINGS
        tco2.index = tco2.index + pd.Timedelta(hours=1)

        # Merging with the original DataFrame M
        device_df = pd.merge(device_df, tco2, on='timestamp', how='left')
        device_df['co2'].fillna(0, inplace=True)
        device_df['co2'] = device_df['kW'] * device_df['co2']
        return device_df

    # Dates broken into 30 day chunks to prevent API error
    def chunk_dates_and_fetch_carbon_data(self, start_date, end_date):
        start_date = datetime.fromisoformat(start_date)
        end_date = datetime.fromisoformat(end_date)

        all_timestamps = []
        all_co2 = []

        # Split date range into chunks of 30 days or less
        while start_date < end_date:
            chunk_end_date = min(start_date + timedelta(days=30), end_date)
            ts, co2 = self.fetch_carbon_data_for_date_range(start_date, chunk_end_date)
            all_timestamps.extend(ts)
            all_co2.extend(co2)
            start_date = chunk_end_date
        
        return all_timestamps, all_co2

    def fetch_carbon_data_for_date_range(self, start_date, end_date):
        # Convert to ISO8601 format and then replace the last characters to fit the required 'Z' format.
        #start_date_iso = datetime.fromisoformat(start_date).isoformat().replace('+00:00', 'Z')
        #end_date_iso = datetime.fromisoformat(end_date).isoformat().replace('+00:00', 'Z')

        start_date_iso = start_date.isoformat().replace('+00:00', 'Z')
        end_date_iso = end_date.isoformat().replace('+00:00', 'Z')

        url = f'https://api.carbonintensity.org.uk/intensity/{start_date_iso}/{end_date_iso}/'
        
        # Make a HTTP request to the URL with retries
        failed_attempts = 0
        max_retries = 5
        for _ in range(max_retries):
            data = requests.get(url, timeout=10)
            # check if the response was successful 
            if(data.status_code == 200):
                data = data.json()
            else:
                print("HTTP request failed. Response code: " + str(data.status_code))
                data = data.json()
                print(data['error']['message'])

                failed_attempts += 1
                if failed_attempts == max_retries:
                    print("Reached maximum retries. Exiting program.")
                    sys.exit(1)

        # Extract useful data from the response
        timestamp = [datetime.fromisoformat(item['from'].replace('Z', '+00:00')).strftime('%Y-%m-%d %H:%M') for item in data['data']]
        co2 = [item['intensity']['actual'] / 1000 / self.tscale for item in data['data']]
        return timestamp, co2
    
class DataPresentation:
    def get_machine_data_paths():
        machine_data_paths = []

        # Define the current directory
        current_directory = os.getcwd()

        # Define valid timing to ensure only valid csvs may be analysed
        timings = ["minutely", "hourly", "daily"]

        for timing in timings:
            search_pattern = os.path.join(current_directory, f"*/{timing}_*.csv")
            for csv_file in glob.glob(search_pattern):
                # Extract the machine name and directory
                machine_name = os.path.basename(csv_file).split("_")[1].replace(".csv", "")
                directory_name = os.path.basename(os.path.dirname(csv_file))
                
                # Check if the directory name matches the machine name
                if machine_name == directory_name:
                    # Format the directory as required and add to the list
                    formatted_directory = f"{machine_name}/{timing}_{machine_name}.csv"
                    machine_data_paths.append(formatted_directory)
        return machine_data_paths

    def calculate_metrics_between_datetimes(selected_csv, from_datetime=None, to_datetime=None):
        #Extracting the timing and device name from the filename
        parts = selected_csv.split('/')
        file_name = parts[1]
        file_parts = file_name.split('_')
        timing = file_parts[0]
        device_name = file_parts[1].replace('.csv', '')

        device_df = pd.read_csv(selected_csv)
        device_df['timestamp'] = pd.to_datetime(device_df['timestamp'])

        # If from_datetime or to_datetime are not provided, set them to the min and max timestamp in the csv.
        if from_datetime is None:
            from_datetime = device_df['timestamp'].min()
        if to_datetime is None:
            to_datetime = device_df['timestamp'].max()

        # Extracting data within the datetime range
        device_data_range = device_df[(device_df['timestamp'] >= from_datetime) & (device_df['timestamp'] <= to_datetime)]

        print("\n----------------------{}--------------------".format(device_name))
        
        total_kW = device_data_range['kW'].sum()
        print("\ntotal kW between {} and {} is = {:.2f} kW".format(from_datetime, to_datetime, total_kW))
        
        if timing.lower() != "minutely":
            total_co2 = device_data_range['co2'].sum()
            print("\ntotal CO2 emissions between {} and {} is = {:.2f} kg".format(from_datetime, to_datetime, total_co2))
        else:
            print('\nCannot estimate CO2 if the timing = "minutely"')

        total_cost = device_data_range['cost'].sum()
        print("\ntotal cost between {} and {} is = £{:.2f}".format(from_datetime, to_datetime, total_cost))
        
        avg_channel1 = device_data_range['channel1'][device_data_range['channel1'] != 0].mean()
        avg_channel2 = device_data_range['channel2'][device_data_range['channel2'] != 0].mean()
        avg_channel3 = device_data_range['channel3'][device_data_range['channel3'] != 0].mean()
        print("\nAverage (non-zero) Current (A) in each channels between {} and {} are = [{:.2f} A, {:.2f} A, {:.2f} A]".format(from_datetime, to_datetime, avg_channel1, avg_channel2, avg_channel3))
        
        total_unbalanced = device_data_range['unbalanced'][device_data_range['unbalanced'] != 0].mean()
        print("\nAverage (non-zero) load imbalance between {} and {} is = {:.2f}%".format(from_datetime, to_datetime, total_unbalanced))

    


# Select machines and fill in additional data to generate minutely hourly or daily csv files
Run this block of code, then input boxes will appear below the code block

In [3]:
# Read the CSV file into a DataFrame
device_df = pd.read_csv("Results.csv")

# Get unique values from the 'Device Name' column
unique_names = device_df['deviceName'].unique()

# Initialize global variables
MACHINE_NAMES = []
VOLTAGE = {}
PHASE = {}
DAY_UNIT_COST = {}
NIGHT_UNIT_COST = {}
NIGHT_TARIFF_START = {}
NIGHT_TARIFF_END = {}
REGION = 'national'
TIMING = 'hourly'

# Generate a list of hourly times
times = [f"{hour:02d}:00" for hour in range(24)]

# Dropdown for 'TIMING'
timing_dropdown = widgets.Dropdown(
    options=['minutely', 'hourly', 'daily'],
    value='hourly',
    description='Timing:',
    style={'description_width': 'initial'}
)

# Multi-selection for 'MACHINE_NAMES'
machine_names_selector = widgets.SelectMultiple(
    options=unique_names,
    description='Machine Names',
    disabled=False,
    style={'description_width': 'initial'}
)

# Output widget to display dynamic form based on selected machine names
output_widget = widgets.Output()

# Update 'TIMING'
def update_timing(change):
    global TIMING
    TIMING = change['new']
    print('TIMING is set to:', TIMING)

# Update 'MACHINE_NAMES' and create form for 'VOLTAGE', 'PHASE', 'NIGHT_TARIFF_START', 'NIGHT_TARIFF_END'
def update_machine_names(change):
    global MACHINE_NAMES
    MACHINE_NAMES = list(change['new'])
    update_form()

def update_voltage(machine, change):
    VOLTAGE[machine] = change['new']

def update_phase(machine, change):
    PHASE[machine] = change['new']

def update_day_unit_cost(machine, change):
    DAY_UNIT_COST[machine] = change['new']

def update_night_unit_cost(machine, change):
    NIGHT_UNIT_COST[machine] = change['new']

def update_night_tariff_start(machine, change):
    NIGHT_TARIFF_START[machine] = change['new']

def update_night_tariff_end(machine, change):
    NIGHT_TARIFF_END[machine] = change['new']

# Function to update the form based on selected machine names
def update_form():
    with output_widget:
        clear_output(wait=True)
        for machine in MACHINE_NAMES:
            print(f'\n\nEnter values for {machine}:')

            # Voltage input
            voltage_input = widgets.FloatText(
                description=f'{machine} Voltage:',
                style={'description_width': 'initial'}
            )
            display(voltage_input)
            
            # Phase input
            phase_input = widgets.Dropdown(
                options=[1, 3],
                description=f'{machine} Phase:',
                style={'description_width': 'initial'}
            )
            display(phase_input)

            # Day unit cost input
            day_unit_cost_input = widgets.FloatText(
                description=f'{machine} Day unit cost:',
                style={'description_width': 'initial'}
            )
            display(day_unit_cost_input)

            # Night unit cost input
            night_unit_cost_input = widgets.FloatText(
                description=f'{machine} Night unit cost:',
                style={'description_width': 'initial'}
            )
            display(night_unit_cost_input)
            
            # Night tariff start time, default set to 22:00
            night_tariff_start_input = widgets.Dropdown(
                options=times,
                value='22:00',
                description=f'{machine} Night Tariff Start:',
                style={'description_width': 'initial'}
            )
            display(night_tariff_start_input)
            
            # Night tariff end time, default set to 08:00
            night_tariff_end_input = widgets.Dropdown(
                options=times,
                value='08:00',
                description=f'{machine} Night Tariff End:',
                style={'description_width': 'initial'}
            )
            display(night_tariff_end_input)
            
            # Attach update function to value changes
            voltage_input.observe(lambda change, machine=machine: update_voltage(machine, change), names='value')
            phase_input.observe(lambda change, machine=machine: update_phase(machine, change), names='value')
            day_unit_cost_input.observe(lambda change, machine=machine: update_day_unit_cost(machine, change), names='value')
            night_unit_cost_input.observe(lambda change, machine=machine: update_night_unit_cost(machine, change), names='value')
            night_tariff_start_input.observe(lambda change, machine=machine: update_night_tariff_start(machine, change), names='value')
            night_tariff_end_input.observe(lambda change, machine=machine: update_night_tariff_end(machine, change), names='value')
            
            # Save widgets for later retrieval of values
            VOLTAGE[machine] = voltage_input.value
            PHASE[machine] = phase_input.value
            DAY_UNIT_COST[machine] = day_unit_cost_input.value
            NIGHT_UNIT_COST[machine] = night_unit_cost_input.value
            NIGHT_TARIFF_START[machine] = night_tariff_start_input.value
            NIGHT_TARIFF_END[machine] = night_tariff_end_input.value

# Attach update functions to widget value changes
timing_dropdown.observe(update_timing, names='value')
machine_names_selector.observe(update_machine_names, names='value')

# Display widgets
display(timing_dropdown)
display(machine_names_selector)
display(output_widget)

Dropdown(description='Timing:', index=1, options=('minutely', 'hourly', 'daily'), style=DescriptionStyle(descr…

SelectMultiple(description='Machine Names', options=('FC-extraction', 'FC-polin', 'FC-monodeck', 'FC-dishwashe…

Output()

TIMING is set to: minutely
TIMING is set to: hourly


# Produce .csv files with analysed data

These will be saved in a directory coresponding with the machine name and timing

In [76]:
if not MACHINE_NAMES:
    print("Warning: no machines are selected")

# Taking user inputs and converting into list of dictionaries that can be parsed into processing functions
machines_user_inputs = []
for machine in MACHINE_NAMES:
    machine_dict = {
        'Machine Name': machine,
        'Voltage': VOLTAGE[machine],
        'Phase': PHASE[machine],
        'Day Unit Cost': DAY_UNIT_COST[machine],
        'Night Unit Cost': NIGHT_UNIT_COST[machine],
        
        # Converting str inputs to ints
        'Night Tariff Start': int(NIGHT_TARIFF_START[machine].split(":")[0]),
        'Night Tariff End': int(NIGHT_TARIFF_END[machine].split(":")[0]),
    }
    machines_user_inputs.append(machine_dict)
    # Checking user has entered entries for all required inputs
    for key, value in machine_dict.items():
        if value == 0:
            print(f"Warning: {machine}'s {key} is set to 0")

# Breaking the data for each machine into its own csv
processor = CTDataProcessor(TIMING)

machines_to_remove = processor.process(MACHINE_NAMES)

# Removing machines with too little data to analyse from the list of user requests
filtered_machines_user_inputs = [machine for machine in machines_user_inputs if machine['Machine Name'] not in machines_to_remove]

# Using data from results combined with user inputs to generate additional data for each selected machine
for machine_user_inputs in filtered_machines_user_inputs:
    analyser = CTDataAnalyser(machine_user_inputs, TIMING, REGION)
    analyser.machine_calculations()

C:\Users\ben.sinclair\AppData\Local\Temp\ipykernel_23064\715307944.py:111: RuntimeWarning:

invalid value encountered in divide

C:\Users\ben.sinclair\AppData\Local\Temp\ipykernel_23064\715307944.py:111: RuntimeWarning:

invalid value encountered in divide



# Plot figures
Run code below to generate figures then use the dropdowns to interact with them. Note that two identical graphs will be generated.

ctrl+click to select multiple columns for display

In [77]:
# Getting all valid paths for csv files that can be plotted
# Searches for any path in form: {machine_name}/{timing}_{machine_name}.csv
machine_data_paths = DataPresentation.get_machine_data_paths()

# Dropdown for selecting CSV file
file_selector = widgets.Dropdown(
    options=machine_data_paths,
    description='CSV File:',
    disabled=False
)

# Multi-selection widget for columns (initialize with an empty list)
column_selector = widgets.SelectMultiple(
    options=[],
    description='Columns',
    disabled=False
)

out = widgets.Output()

# Getting options for column dropdown
def update_columns(change):
    df = pd.read_csv(file_selector.value, parse_dates=["timestamp"])
    column_selector.options = df.columns[1:]

file_selector.observe(update_columns, names='value')
update_columns(None)

# Define a plotting function with a secondary Y-axis
def plot_data(file, columns):
    with out:
        out.clear_output(wait=True)  # Clear the previous graph

        df = pd.read_csv(file, parse_dates=["timestamp"])

        # Determine which columns to put on the secondary axis
        secondary_y_columns = [col for col in columns if df[col].std() > 10]

        # Initialize figure
        fig = go.Figure()

        # Add traces
        for col in columns:
            fig.add_trace(go.Scatter(x=df['timestamp'], y=df[col], name=col, yaxis='y2' if col in secondary_y_columns else 'y'))

        # Update layout for secondary axis
        fig.update_layout(
            yaxis2=dict(
                overlaying='y',
                side='right'
            )
        )
        fig.show()
        out.clear_output(wait=True)
        print("")

display(out)

# Use interactive widget to show the graph
widgets.interactive(plot_data, file=file_selector, columns=column_selector)

Output()

interactive(children=(Dropdown(description='CSV File:', options=('FC-extraction/minutely_FC-extraction.csv', '…

# Calculate total kW between datetime
Modify the datetimes below and then run code to calculate results (range is inclusive of both entered datetimes). Default values for selected csv will be the first and last datetimes.

In [78]:
# Getting all valid paths for csv files that can be plotted
machine_data_paths = DataPresentation.get_machine_data_paths()

# Dropdown for selecting CSV file
file_selector = widgets.Dropdown(
    options=machine_data_paths,
    description='CSV File:',
    disabled=False,
    value = None
)

# Global variables to store from_datetime and to_datetime
global from_datetime, to_datetime
global selected_csv
from_datetime, to_datetime = None, None
selected_csv = None

def update_datetime(*args):
    global from_datetime, to_datetime
    try:
        from_date_str = date_selector_start.value.strftime('%Y-%m-%d')
        from_time_str = time_selector_start.value
        to_date_str = date_selector_end.value.strftime('%Y-%m-%d')
        to_time_str = time_selector_end.value

        from_datetime = pd.to_datetime(f'{from_date_str} {from_time_str}')
        to_datetime = pd.to_datetime(f'{to_date_str} {to_time_str}')

        # print(f'From Datetime: {from_datetime}')
        # print(f'To Datetime: {to_datetime}')
    except Exception as e:
        print(f"An error occurred: {e}")

def load_csv(change):
    global selected_csv
    if change['type'] == 'change' and change['name'] == 'value':
        selected_csv = change['new']
        try:
            # Load the CSV file
            df = pd.read_csv(change['new'])

            # Convert the datetime column to datetime objects
            df['timestamp'] = pd.to_datetime(df['timestamp'])

            # Get the first and last datetime
            start_datetime = df['timestamp'].min()
            end_datetime = df['timestamp'].max()

            # Update the datetime selectors
            date_selector_start.value = start_datetime.date()
            time_selector_start.value = start_datetime.strftime('%H:%M')
            date_selector_end.value = end_datetime.date()
            time_selector_end.value = end_datetime.strftime('%H:%M')
        except Exception as e:
            print(f"An error occurred: {e}")

# Create date and time selectors
date_selector_start = widgets.DatePicker(value=datetime.now() - timedelta(days=1), description='Start Date', disabled=False)
time_selector_start = widgets.Text(value='00:00', description='Start Time')
date_selector_end = widgets.DatePicker(value=datetime.now(), description='End Date', disabled=False)
time_selector_end = widgets.Text(value='23:59', description='End Time')

# Function to get datetime from date and time widgets
def get_datetime(date_widget, time_widget):
    date_str = date_widget.value.strftime('%Y-%m-%d')
    time_str = time_widget.value
    return pd.to_datetime(f'{date_str} {time_str}')

file_selector.observe(load_csv)

# Set update_datetime to be called whenever any date or time widget value changes
date_selector_start.observe(update_datetime, 'value')
time_selector_start.observe(update_datetime, 'value')
date_selector_end.observe(update_datetime, 'value')
time_selector_end.observe(update_datetime, 'value')

# Display widgets
display(file_selector)
display(date_selector_start, time_selector_start)
display(date_selector_end, time_selector_end)

# Use get_datetime function to get start and end datetime for the next function
start_datetime = get_datetime(date_selector_start, time_selector_start)
end_datetime = get_datetime(date_selector_end, time_selector_end)

Dropdown(description='CSV File:', options=('FC-extraction/minutely_FC-extraction.csv', 'FC-extraction/hourly_F…

DatePicker(value=datetime.datetime(2024, 1, 8, 16, 47, 47, 631353), description='Start Date', step=1)

Text(value='00:00', description='Start Time')

DatePicker(value=datetime.datetime(2024, 1, 9, 16, 47, 47, 633353), description='End Date', step=1)

Text(value='23:59', description='End Time')

In [81]:
# will modify this so a from_datetime and a to_datetime is passed in with the file nane selected
DataPresentation.calculate_metrics_between_datetimes(selected_csv, from_datetime, to_datetime)


----------------------FC-extraction--------------------

total kW between 2023-10-12 17:00:00 and 2023-11-01 14:00:00 is = 372.73 kW

total CO2 emissions between 2023-10-12 17:00:00 and 2023-11-01 14:00:00 is = 61.91 kg

total cost between 2023-10-12 17:00:00 and 2023-11-01 14:00:00 is = £69.34

Average (non-zero) Current (A) in each channels between 2023-10-12 17:00:00 and 2023-11-01 14:00:00 are = [nan A, nan A, nan A]

Average (non-zero) load imbalance between 2023-10-12 17:00:00 and 2023-11-01 14:00:00 is = nan%


## Select machines to generate dataframes
If using deepnote this will eneable deepnote graphing cells to be used

In [9]:
global machine_data_paths
machine_data_paths = DataPresentation.get_machine_data_paths()

# Multi-selection for 'machine_data_paths'
machine_names_selector = widgets.SelectMultiple(
    options=machine_data_paths,
    description='Machine Names',
    disabled=False,
    style={'description_width': 'initial'}
)

# Update 'MACHINE_NAMES'
def update_machine_names(change):
    global machine_data_paths
    machine_data_paths = list(change['new'])
    create_dataframes_for_machines(machine_data_paths)

def create_dataframes_for_machines(machine_data_paths):
    for machine_path in machine_data_paths:
        parts = machine_path.split('/')
        machine = parts[0]
        df_name = f"{machine}_df"
        globals()[df_name] = pd.read_csv(machine_path)

machine_names_selector.observe(update_machine_names, names='value')

display(machine_names_selector)

SelectMultiple(description='Machine Names', options=('EnvTestChamber/hourly_EnvTestChamber.csv',), style=Descr…